In [1]:
import geopandas as gpd

In [3]:
import os

In [2]:
path = r'T:\Trans Projects\Model Development\UrbanSim_LandUse\Output\Simulation_47_Final_RTP'

In [4]:
shp = 'parcel_fabric.shp'

In [5]:
parcel_data = gpd.read_file(os.path.join(path, shp))

In [6]:
parcel_data.columns

Index(['btype', 'nrsqft', 'rsqft', 'du', 'yrbuilt', 'lpid', 'pundev',
       'dev_land', 'developed', 'obtype', 'orsqft', 'onrsqft', 'odu', 'zid',
       'rezoned', 'city', 'annexed', 'ozid', 'geometry'],
      dtype='object')

In [7]:
parcel_data.head()

,btype,nrsqft,rsqft,du,yrbuilt,lpid,pundev,dev_land,developed,obtype,orsqft,onrsqft,odu,zid,rezoned,city,annexed,ozid,geometry
0,(),0.0,0.0,0.0,0.0,24284,0.00,1.123953e+04,0,(),0.0,0.0,0.0,119,0,None,0,119,"MULTIPOLYGON (((4254437.869 843733.606, 425444..."
1,"(5100,)",1662.0,0.0,0.0,1969.0,24356,0.15,1.973955e+06,0,"(5100,)",0.0,1662.0,0.0,119,0,None,0,119,"POLYGON ((4254479.021 845048.443, 4254437.869 ..."
2,"(1110,)",0.0,3468.0,1.0,2006.0,24391,0.00,1.099604e+05,0,"(1110,)",3468.0,0.0,1.0,62,0,None,0,62,"POLYGON ((4250857.967 845652.695, 4250847.053 ..."
3,"(1110,)",0.0,1296.0,1.0,1984.0,24408,0.00,6.144933e+04,0,"(1110,)",1296.0,0.0,1.0,62,0,None,0,62,"POLYGON ((4250865.647 845846.211, 4250859.554 ..."
4,"(1110,)",0.0,2528.0,1.0,2015.0,24419,0.00,6.097905e+04,0,"(1110,)",2528.0,0.0,1.0,62,0,None,0,62,"POLYGON ((4250871.702 845998.498, 4250865.647 ..."


In [9]:
parcel_data.shape

(78216, 19)